In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization, Dropout
from datetime import datetime , timedelta
import keras_tuner as kt


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import torch
print(torch.cuda.is_available())

True


In [4]:

# Define parameters
prediction_days = 120

# Download stock data
start_date=  datetime.now() - timedelta(days=365)
end_date=datetime.today()

stock_data = yf.download('TATASTEEL.NS', start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [5]:


# Prepare training data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(stock_data['Close'].values.reshape(-1, 1))

x_train = []
y_train = []

for x in range(prediction_days, len(scaled_data)):
    x_train.append(scaled_data[x - prediction_days:x, 0])
    y_train.append(scaled_data[x, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


In [6]:

# Define the model building function
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units_1', min_value=50, max_value=100, step=10),
                   return_sequences=True,
                   input_shape=(x_train.shape[1], 1)))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))
    
    model.add(LSTM(units=hp.Int('units_2', min_value=50, max_value=100, step=10),
                   return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)))
    
    model.add(LSTM(units=hp.Int('units_3', min_value=50, max_value=100, step=10)))
    model.add(Dropout(hp.Float('dropout_3', min_value=0.1, max_value=0.5, step=0.1)))
    
    model.add(Dense(units=25, activation='relu'))
    model.add(Dense(units=1))
    
    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop']),
                  loss='mean_squared_error')
     
    # Move Model to GPU
    #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #model.to(device)
    
    return model


In [7]:

# Set up Keras Tuner
tuner = kt.Hyperband(build_model,
                     objective='val_loss',
                     max_epochs=25,
                     hyperband_iterations=2,
                     directory='my_dir',
                     project_name='stock_price_tuning')


Reloading Tuner from my_dir\stock_price_tuning\tuner0.json


In [8]:

# Split the data for validation
split = int(0.8 * len(x_train))
x_train_split, x_val_split = x_train[:split], x_train[split:]
y_train_split, y_val_split = y_train[:split], y_train[split:]


In [9]:

# Search for the best hyperparameters
tuner.search(x_train_split, y_train_split,
             epochs=25,
             batch_size=32,
             validation_data=(x_val_split, y_val_split))



In [10]:
# Retrieve the best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters()[0]

# Build and compile the best model
best_model = build_model(best_hyperparameters)

# Train the best model on the full training data
best_model.fit(x_train, y_train, epochs=25, batch_size=32)

Epoch 1/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 102ms/step - loss: 0.4795
Epoch 2/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - loss: 0.2683
Epoch 3/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - loss: 0.1251
Epoch 4/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - loss: 0.1164
Epoch 5/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - loss: 0.0870
Epoch 6/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - loss: 0.0945
Epoch 7/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - loss: 0.0542
Epoch 8/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - loss: 0.0602
Epoch 9/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - loss: 0.0594
Epoch 10/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - loss: 0.0653
Epoch 11/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - loss: 0.0504
Epoch 12/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - loss: 0.0566
Epoch 13/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - loss: 0.0624
Epoch 14/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - loss: 0.0509
Epoch 15/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - loss: 0.0458
Epoch 16/25
4/4 ━━━━━━━━━━━━━━━

In [11]:

# Initialize the scaler
scaler = MinMaxScaler()

# Fit scaler only on training data
scaler.fit(stock_data['Close'].values.reshape(-1, 1))

# Normalize training data
scaled_train_data = scaler.transform(stock_data['Close'].values.reshape(-1, 1))

# Prepare test data (make sure to use the same scaler)
test_data = yf.download('TATASTEEL.NS', start=start_date, end=end_date)
actual_prices = test_data['Close'].values

# Normalize test data using the same scaler
scaled_test_data = scaler.transform(test_data['Close'].values.reshape(-1, 1))

# Combine and prepare model inputs as before
total_dataset = np.concatenate((scaled_train_data, scaled_test_data), axis=0)
model_inputs = total_dataset[len(total_dataset) - len(scaled_test_data) - prediction_days:].reshape(-1, 1)

# Prepare x_test for prediction (same as before)
x_test = []
for x in range(prediction_days, len(model_inputs)):
    x_test.append(model_inputs[x - prediction_days:x, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# Make prediction for tomorrow
predicted_price = best_model.predict(x_test[-1].reshape(1, prediction_days, 1))


[*********************100%%**********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 502ms/step


In [12]:

# Make predictions
predicted_prices = best_model.predict(x_test)
predicted_prices = scaler.inverse_transform(predicted_prices)

# Get the last prediction
real_data = model_inputs[-prediction_days:].reshape(1, prediction_days, 1)
prediction = best_model.predict(real_data)
prediction = scaler.inverse_transform(prediction)

# Print results
print(f"Last actual closing price: {actual_prices[-1]}")
print(f"Last predicted closing price: {prediction[0][0]}")


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Last actual closing price: 164.36000061035156
Last predicted closing price: 144.41943359375


In [13]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Assuming `predicted_prices` and `actual_prices` are your predictions and true values

# Calculate MAE, MSE, and RMSE
mae = mean_absolute_error(actual_prices[-len(predicted_prices):], predicted_prices)
mse = mean_squared_error(actual_prices[-len(predicted_prices):], predicted_prices)
rmse = np.sqrt(mse)

# Calculate MAPE
actual_values = actual_prices[-len(predicted_prices):]
mape = np.mean(np.abs((actual_values - predicted_prices.flatten()) / actual_values)) * 100

# Calculate accuracy percentage
accuracy_percentage = 100 - mape

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"Accuracy Percentage: {accuracy_percentage:.2f}%")


Mean Absolute Error: 13.606145577352555
Mean Squared Error: 264.42942261080054
Root Mean Squared Error: 16.261286007287385
Mean Absolute Percentage Error (MAPE): 8.81%
Accuracy Percentage: 91.19%


In [22]:
# Save the entire model
best_model.save("D:\Projects\Investment _Prediction\Jupyter NoteBook\lstm_model.h5")
